In [1]:
import numpy as np
import pandas as pd

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 250)

pd.options.display.float_format = '{:.0f}'.format

/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
class Fixer:
    
    # class variables
    classes = ['rf', 'fr', 'so', 'jr', 'sr']
    prefixes = ['rf_', 'fr_', 'so_', 'jr_', 'sr_']
    empty_stats_year = {'year': np.nan, 'position': np.nan, 'games': np.nan, 'tackles_solo': np.nan, 'tackles_asst': np.nan, 
                        'tfl_solo': np.nan, 'tfl_asst': np.nan, 'tfl_yards': np.nan, 'sacks_solo': np.nan, 'sacks_asst': np.nan, 
                        'sacks_yards': np.nan, 'int': np.nan, 'int_yards': np.nan, 'int_td': np.nan, 'fum': np.nan, 
                        'fum_yards': np.nan, 'fum_td': np.nan, 'ffum': np.nan, 'safety': np.nan, 'punt_ret': np.nan, 'punt_ret_yards': np.nan, 
                        'punt_ret_td': np.nan, 'kick_ret': np.nan, 'kick_ret_yards': np.nan, 'kick_ret_td': np.nan, 'total_points': np.nan}
    agg_stats = ['games', 'tackles_solo', 'tackles_asst', 'tfl_solo', 'tfl_asst', 'tfl_yards', 
                 'sacks_solo', 'sacks_asst', 'sacks_yards', 'int', 'int_yards', 'int_td', 'fum', 'fum_yards', 
                 'fum_td', 'ffum', 'safety', 'punt_ret', 'punt_ret_yards', 'punt_ret_td', 'kick_ret', 'kick_ret_yards',
                 'kick_ret_td', 'total_points']
    
    extracted_dups = 0
    
    def __init__(self, team_name):
        self.file_name = team_name + '.csv'
        self.df = pd.read_csv(self.file_name, index_col = 0)

              
    def pprintDF(self):
        return self.df[['player_id', 'player', 'ncaa_yr_ct', 'team', 'games', 'tackles_solo',
                        'rf_year', 'rf_games', 'rf_tackles_solo', 
                        'fr_year', 'fr_games', 'fr_tackles_solo',
                        'so_year', 'so_games', 'so_tackles_solo',
                        'jr_year', 'jr_games', 'jr_tackles_solo',
                        'sr_year', 'sr_games', 'sr_tackles_solo']]
    
    def pprint_dups(self):
        return self.dup_players[['player_id', 'player', 'ncaa_yr_ct', 'team', 'games', 'tackles_solo',
                        'rf_year', 'rf_games', 'rf_tackles_solo', 
                        'fr_year', 'fr_games', 'fr_tackles_solo',
                        'so_year', 'so_games', 'so_tackles_solo',
                        'jr_year', 'jr_games', 'jr_tackles_solo',
                        'sr_year', 'sr_games', 'sr_tackles_solo']]


    def extract_dups(self):
        if self.extracted_dups == 0:
            self.dup_players = self.df[self.df['names'].duplicated(keep=False)]
            self.dup_players = self.dup_players.sort_values(by=['names'])
            self.df.drop_duplicates(['names'], keep=False, inplace=True)
            self.df.reset_index(drop=True, inplace=True)
            self.extracted_dups = 1
        return self.dup_players
    
    
    def aggregate_stats(self, player_df):
        for s in self.agg_stats:
            player_df[s] = player_df[[c for c in player_df.columns if c.endswith(s)]].sum(axis = 1, skipna=True)
        player_df['ncaa_yr_ct'] = player_df[[c for c in player_df.columns if c.endswith('year')]].count(axis = 1, numeric_only=True)
        return player_df
        
        
    def reset_order(self, player_df):
        
        gen_cols = ['player_id', 'player', 'names', 'team', 'ncaa_yr_ct']
        
        cols = player_df.columns.tolist()

        # get list of ints for columns with prefixes
        rf = [i for i, e in enumerate(cols) if 'rf_' in e]
        fr = [i for i, e in enumerate(cols) if 'fr_' in e]
        so = [i for i, e in enumerate(cols) if 'so_' in e]
        jr = [i for i, e in enumerate(cols) if 'jr_' in e]
        sr = [i for i, e in enumerate(cols) if 'sr_' in e]

        ordered = gen_cols + self.agg_stats + cols[rf[0]:rf[-1]+1] + cols[fr[0]:fr[-1]+1] + cols[so[0]:so[-1]+1] + cols[jr[0]:jr[-1]+1] + cols[sr[0]:sr[-1]+1]
        
        player_df = player_df[ordered]
        return player_df
    
    
    def append(self, new_player):
        df_count = len(self.df)
        self.df = self.df.append(new_player, ignore_index = True, sort=False)
        self.df.reset_index(drop=True, inplace=True)
        df_count_new = len(self.df)
        print('Appended: {}.  Count was {}.  Count is now {}.'.format(new_player.player, df_count, df_count_new))
    
    
    # one or more years overlap, need to shift years
    # find length, doubles and holes
    def shift(self, stats_dicts):
        # find number of years of stats
        year_count = len(stats_dicts)
        
        # separate dicts into nan and used
        empty = []
        used = []
        top_class_idx = self.classes.index('rf')
        
        for l in stats_dicts:
            for k, v in l.items():
                
                if k.endswith('year'):
                    c, y = k.split('_')
                    if np.isnan(v):
                        empty.append(l)
                    else:
                        used.append(l)
                        tc = self.classes.index(c)
                        if tc > top_class_idx:
                            top_class_idx = tc
                        
        
        
        if len(used) > 5:
            used.remove(used[0])
        
        # select classes to be filled by slicing all possible classes
               
        # select all classes from rf up to the highest class listed
        classes_below_top_class = self.classes[:top_class_idx + 1]

        
        # start at the highest class listed and work back the number of year of stats we have to fill
        classes_to_fill = classes_below_top_class[-len(used):]
        
        
        fixed_stats = []
        used_idx = 0
        
        for cls in self.classes:
            yr_dict = {}
            prefix = cls + '_'
            
            # create a copy of the empty dict, prefix all of the keys and save it on
            if cls not in classes_to_fill:
                for k, v in self.empty_stats_year.items():
                    yr_dict[prefix + k] = v
                fixed_stats.append(yr_dict)
            
            else:
                old_dict = used[used_idx]
                for k in self.empty_stats_year.keys():
                    yr_dict[prefix + k] = [val for (key, val) in old_dict.items() if key.endswith(k)][0]
                fixed_stats.append(yr_dict)
                used_idx += 1
        print('EMPTY: {}'.format(len(empty)))
        print('USED: {}'.format(len(used)))
        print('Classes to fill: {}'.format(classes_to_fill))
        
        return fixed_stats
       
    
    def resolve_top(self, dups_pair):
        stats = []
        shift = 0

        new_record = {}
        cols = dups_pair.columns.tolist()
        
        for c in cols[:5]:
            new_record[c] = dups_pair.iloc[0, cols.index(c)]

        for cls in self.classes:
            yr_dict = {}
            alt_yr_dict = {}
            
            filter_col = [c for c in dups_pair if c.startswith(cls + '_')]
            a = dups_pair.loc[:, filter_col].values
            if (np.isfinite(a[0][0])) & (np.isfinite(a[1][0])):
                yr_dict = dict(zip(filter_col, a[0]))
                alt_yr_dict = dict(zip(filter_col, a[1]))
                shift = 1
            elif np.isfinite(a[0][0]):
                yr_dict = dict(zip(filter_col, a[0]))
            else:
                yr_dict = dict(zip(filter_col, a[1]))

            # collect yr dicts into list
            stats.append(yr_dict)
            if bool(alt_yr_dict):
                stats.append(alt_yr_dict)

        if shift == 1:
            stats = self.shift(stats)
        
        # add keys and values from each year's stats to new_record
        for y in stats:
            for k, v in y.items():
                new_record[k] = v
        
        
        self.dup_players.drop(index=dups_pair.index, inplace=True)
        new_player = pd.DataFrame(new_record, index=[0])
        new_player = self.aggregate_stats(new_player)
        new_player = self.reset_order(new_player)
        self.append(new_player)
        
        
    # this is the default case where years for the two entries don't overlap
    def resolve_bottom(self, dups_pair):
        stats = []
        shift = 0

        new_record = {}
        cols = dups_pair.columns.tolist()
        
        for c in cols[:5]:
            new_record[c] = dups_pair.iloc[1, cols.index(c)]

        for cls in self.classes:
            yr_dict = {}
            alt_yr_dict = {}
            
            filter_col = [c for c in dups_pair if c.startswith(cls + '_')]
            a = dups_pair.loc[:, filter_col].values
            if (np.isfinite(a[0][0])) & (np.isfinite(a[1][0])):
                yr_dict = dict(zip(filter_col, a[0]))
                alt_yr_dict = dict(zip(filter_col, a[1]))
                shift = 1
            elif np.isfinite(a[0][0]):
                yr_dict = dict(zip(filter_col, a[0]))
            else:
                yr_dict = dict(zip(filter_col, a[1]))

            # collect yr dicts into list
            stats.append(yr_dict)
            if bool(alt_yr_dict):
                stats.append(alt_yr_dict)

        if shift == 1:
            stats = self.shift(stats)
        
        # add keys and values from each year's stats to new_record
        for y in stats:
            for k, v in y.items():
                new_record[k] = v
        
        
        self.dup_players.drop(index=dups_pair.index, inplace=True)
        new_player = pd.DataFrame(new_record, index=[0])
        new_player = self.aggregate_stats(new_player)
        new_player = self.reset_order(new_player)
        self.append(new_player)
    
    def skip(self, dups_pair):
        self.append(dups_pair)
        self.dup_players.drop(index=dups_pair.index, inplace=True)
        self.dup_players.reset_index(drop=True, inplace=True)
        print(len(self.dup_players))

In [3]:
fx = Fixer('liberty')
fx.df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 96 entries, 0 to 95
Columns: 159 entries, player_id to total_points
dtypes: float64(149), int64(2), object(8)
memory usage: 120.0+ KB


In [ ]:
fx.extracted_dups = 0

In [6]:
# extract dups
fx.extract_dups();
fx.dup_players = fx.dup_players.reset_index(drop=True);
fx.dup_players

,player_id,player,names,team,ncaa_yr_ct,rf_year,rf_position,rf_games,rf_tackles_solo,rf_tackles_asst,rf_tfl_solo,rf_tfl_asst,rf_tfl_yards,rf_sacks_solo,rf_sacks_asst,rf_sacks_yards,rf_int,rf_int_yards,rf_int_td,rf_fum,rf_fum_yards,rf_fum_td,rf_ffum,rf_safety,rf_punt_ret,rf_punt_ret_yards,rf_punt_ret_td,rf_kick_ret,rf_kick_ret_yards,rf_kick_ret_td,rf_total_points,fr_year,fr_position,fr_games,fr_tackles_solo,fr_tackles_asst,fr_tfl_solo,fr_tfl_asst,fr_tfl_yards,fr_sacks_solo,fr_sacks_asst,fr_sacks_yards,fr_int,fr_int_yards,fr_int_td,fr_fum,fr_fum_yards,fr_fum_td,fr_ffum,fr_safety,fr_punt_ret,fr_punt_ret_yards,fr_punt_ret_td,fr_kick_ret,fr_kick_ret_yards,fr_kick_ret_td,fr_total_points,so_year,so_position,so_games,so_tackles_solo,so_tackles_asst,so_tfl_solo,so_tfl_asst,so_tfl_yards,so_sacks_solo,so_sacks_asst,so_sacks_yards,so_int,so_int_yards,so_int_td,so_fum,so_fum_yards,so_fum_td,so_ffum,so_safety,so_punt_ret,so_punt_ret_yards,so_punt_ret_td,so_kick_ret,so_kick_ret_yards,so_kick_ret_td,so_total_points,jr_year,jr_position,jr_games,jr_tackles_solo,jr_tackles_asst,jr_tfl_solo,jr_tfl_asst,jr_tfl_yards,jr_sacks_solo,jr_sacks_asst,jr_sacks_yards,jr_int,jr_int_yards,jr_int_td,jr_fum,jr_fum_yards,jr_fum_td,jr_ffum,jr_safety,jr_punt_ret,jr_punt_ret_yards,jr_punt_ret_td,jr_kick_ret,jr_kick_ret_yards,jr_kick_ret_td,jr_total_points,sr_year,sr_position,sr_games,sr_tackles_solo,sr_tackles_asst,sr_tfl_solo,sr_tfl_asst,sr_tfl_yards,sr_sacks_solo,sr_sacks_asst,sr_sacks_yards,sr_int,sr_int_yards,sr_int_td,sr_fum,sr_fum_yards,sr_fum_td,sr_ffum,sr_safety,sr_punt_ret,sr_punt_ret_yards,sr_punt_ret_td,sr_kick_ret,sr_kick_ret_yards,sr_kick_ret_td,sr_total_points,games,tackles_solo,tackles_asst,tfl_solo,tfl_asst,tfl_yards,sacks_solo,sacks_asst,sacks_yards,int,int_yards,int_td,fum,fum_yards,fum_td,ffum,safety,punt_ret,punt_ret_yards,punt_ret_td,kick_ret,kick_ret_yards,kick_ret_td,total_points


,player_id,player,names,team,ncaa_yr_ct,rf_year,rf_position,rf_games,rf_tackles_solo,rf_tackles_asst,rf_tfl_solo,rf_tfl_asst,rf_tfl_yards,rf_sacks_solo,rf_sacks_asst,rf_sacks_yards,rf_int,rf_int_yards,rf_int_td,rf_fum,rf_fum_yards,rf_fum_td,rf_ffum,rf_safety,rf_punt_ret,rf_punt_ret_yards,rf_punt_ret_td,rf_kick_ret,rf_kick_ret_yards,rf_kick_ret_td,rf_total_points,fr_year,fr_position,fr_games,fr_tackles_solo,fr_tackles_asst,fr_tfl_solo,fr_tfl_asst,fr_tfl_yards,fr_sacks_solo,fr_sacks_asst,fr_sacks_yards,fr_int,fr_int_yards,fr_int_td,fr_fum,fr_fum_yards,fr_fum_td,fr_ffum,fr_safety,fr_punt_ret,fr_punt_ret_yards,fr_punt_ret_td,fr_kick_ret,fr_kick_ret_yards,fr_kick_ret_td,fr_total_points,so_year,so_position,so_games,so_tackles_solo,so_tackles_asst,so_tfl_solo,so_tfl_asst,so_tfl_yards,so_sacks_solo,so_sacks_asst,so_sacks_yards,so_int,so_int_yards,so_int_td,so_fum,so_fum_yards,so_fum_td,so_ffum,so_safety,so_punt_ret,so_punt_ret_yards,so_punt_ret_td,so_kick_ret,so_kick_ret_yards,so_kick_ret_td,so_total_points,jr_year,jr_position,jr_games,jr_tackles_solo,jr_tackles_asst,jr_tfl_solo,jr_tfl_asst,jr_tfl_yards,jr_sacks_solo,jr_sacks_asst,jr_sacks_yards,jr_int,jr_int_yards,jr_int_td,jr_fum,jr_fum_yards,jr_fum_td,jr_ffum,jr_safety,jr_punt_ret,jr_punt_ret_yards,jr_punt_ret_td,jr_kick_ret,jr_kick_ret_yards,jr_kick_ret_td,jr_total_points,sr_year,sr_position,sr_games,sr_tackles_solo,sr_tackles_asst,sr_tfl_solo,sr_tfl_asst,sr_tfl_yards,sr_sacks_solo,sr_sacks_asst,sr_sacks_yards,sr_int,sr_int_yards,sr_int_td,sr_fum,sr_fum_yards,sr_fum_td,sr_ffum,sr_safety,sr_punt_ret,sr_punt_ret_yards,sr_punt_ret_td,sr_kick_ret,sr_kick_ret_yards,sr_kick_ret_td,sr_total_points,games,tackles_solo,tackles_asst,tfl_solo,tfl_asst,tfl_yards,sacks_solo,sacks_asst,sacks_yards,int,int_yards,int_td,fum,fum_yards,fum_td,ffum,safety,punt_ret,punt_ret_yards,punt_ret_td,kick_ret,kick_ret_yards,kick_ret_td,total_points


In [5]:
fx.dup_players.head(300)

,player_id,player,names,team,ncaa_yr_ct,rf_year,rf_position,rf_games,rf_tackles_solo,rf_tackles_asst,rf_tfl_solo,rf_tfl_asst,rf_tfl_yards,rf_sacks_solo,rf_sacks_asst,rf_sacks_yards,rf_int,rf_int_yards,rf_int_td,rf_fum,rf_fum_yards,rf_fum_td,rf_ffum,rf_safety,rf_punt_ret,rf_punt_ret_yards,rf_punt_ret_td,rf_kick_ret,rf_kick_ret_yards,rf_kick_ret_td,rf_total_points,fr_year,fr_position,fr_games,fr_tackles_solo,fr_tackles_asst,fr_tfl_solo,fr_tfl_asst,fr_tfl_yards,fr_sacks_solo,fr_sacks_asst,fr_sacks_yards,fr_int,fr_int_yards,fr_int_td,fr_fum,fr_fum_yards,fr_fum_td,fr_ffum,fr_safety,fr_punt_ret,fr_punt_ret_yards,fr_punt_ret_td,fr_kick_ret,fr_kick_ret_yards,fr_kick_ret_td,fr_total_points,so_year,so_position,so_games,so_tackles_solo,so_tackles_asst,so_tfl_solo,so_tfl_asst,so_tfl_yards,so_sacks_solo,so_sacks_asst,so_sacks_yards,so_int,so_int_yards,so_int_td,so_fum,so_fum_yards,so_fum_td,so_ffum,so_safety,so_punt_ret,so_punt_ret_yards,so_punt_ret_td,so_kick_ret,so_kick_ret_yards,so_kick_ret_td,so_total_points,jr_year,jr_position,jr_games,jr_tackles_solo,jr_tackles_asst,jr_tfl_solo,jr_tfl_asst,jr_tfl_yards,jr_sacks_solo,jr_sacks_asst,jr_sacks_yards,jr_int,jr_int_yards,jr_int_td,jr_fum,jr_fum_yards,jr_fum_td,jr_ffum,jr_safety,jr_punt_ret,jr_punt_ret_yards,jr_punt_ret_td,jr_kick_ret,jr_kick_ret_yards,jr_kick_ret_td,jr_total_points,sr_year,sr_position,sr_games,sr_tackles_solo,sr_tackles_asst,sr_tfl_solo,sr_tfl_asst,sr_tfl_yards,sr_sacks_solo,sr_sacks_asst,sr_sacks_yards,sr_int,sr_int_yards,sr_int_td,sr_fum,sr_fum_yards,sr_fum_td,sr_ffum,sr_safety,sr_punt_ret,sr_punt_ret_yards,sr_punt_ret_td,sr_kick_ret,sr_kick_ret_yards,sr_kick_ret_td,sr_total_points,games,tackles_solo,tackles_asst,tfl_solo,tfl_asst,tfl_yards,sacks_solo,sacks_asst,sacks_yards,int,int_yards,int_td,fum,fum_yards,fum_td,ffum,safety,punt_ret,punt_ret_yards,punt_ret_td,kick_ret,kick_ret_yards,kick_ret_td,total_points


In [ ]:

fx.dup_players.info()
fx.dup_players.tail(50)

In [ ]:
fx.dup_players.drop(fx.dup_players.index[761:], axis=0, inplace=True)
fx.dup_players.tail(50)

In [ ]:
fx.pprint_dups().head(10)

In [ ]:
p = fx.dup_players.drop([0], axis=0)
# fx.dup_players.drop([p])
print(p)

In [ ]:
fx.skip(fx.dup_players.iloc[:1, :])
fx.pprint_dups().head(10)

In [ ]:
fx.resolve_top(fx.dup_players.iloc[:2, :])
fx.pprint_dups().head(10)
fx.pprintDF().tail()

In [ ]:
fx.resolve_bottom(fx.dup_players.iloc[:2, :])
fx.pprint_dups().head(10)
fx.pprintDF().tail(10)

In [ ]:
fx.dup_players.drop([0], axis=0, inplace=True);
fx.pprint_dups().head(10)

In [ ]:
fx.pprint_dups().head(10)

In [ ]:
fx.df.tail()

In [ ]:
fx.df.to_csv('ncaa_deduped.csv')